In [1]:
#загрузим основные библиотеки
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, StackingRegressor
from sklearn.preprocessing import StandardScaler
 
from sklearn.datasets import fetch_california_housing
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OrdinalEncoder

from sklearn.linear_model import RidgeCV

from sklearn.tree import DecisionTreeRegressor

In [2]:
df_wine = pd.read_csv('data/Red.csv')
df_wine.head()

,Name,Country,Region,Winery,Rating,NumberOfRatings,Price,Year
0,Pomerol 2011,France,Pomerol,Château La Providence,4.2,100,95.00,2011
1,Lirac 2017,France,Lirac,Château Mont-Redon,4.3,100,15.50,2017
2,Erta e China Rosso di Toscana 2015,Italy,Toscana,Renzo Masi,3.9,100,7.45,2015
3,Bardolino 2019,Italy,Bardolino,Cavalchina,3.5,100,8.72,2019
4,Ried Scheibner Pinot Noir 2016,Austria,Carnuntum,Markowitsch,3.9,100,29.15,2016


In [3]:
df_wine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8666 entries, 0 to 8665
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             8666 non-null   object 
 1   Country          8666 non-null   object 
 2   Region           8666 non-null   object 
 3   Winery           8666 non-null   object 
 4   Rating           8666 non-null   float64
 5   NumberOfRatings  8666 non-null   int64  
 6   Price            8666 non-null   float64
 7   Year             8666 non-null   object 
dtypes: float64(2), int64(1), object(5)
memory usage: 541.8+ KB


Задание 6.1

На основании датасета предсказания рейтинга красного вина, с которым вы работали в разделе о предобработке данных в предыдущем юните (файл Red.csv ), вам предстоит выполнить следующее:

Добавить в пайплайн, созданный вами в предыдущем юните, обработку столбца 'Region'. Кодирование 'Region' произвести с использованием OrdinalEncoder. Стандартизацию столбца 'Price' и кодирование столбца 'Country' осуществить в соответствии с примером, представленным в юните 5.
Важно! Для совпадения результатов процесс трансформации столбцов должен выполняться в следующей последовательности:

Кодирование столбца 'Region' через OrdinalEncoder.
Стандартизация столбца 'Price'.
Кодирование столбца 'Country' через OneHotEncoder.
Обучить пайплайн на тренировочном наборе данных (файл Red.csv ) и оценить качество модели по метрике RMSE на тестовом наборе (файл Red_test.csv ).
Зафиксировать random_state=42.
Сохранить пайплайн в файл pipeline_wine.pkl.
В качестве ответа на задание введите в поле ниже полученный результат по метрике RMSE, округленный до четвёртого знака после запятой.

In [4]:
ct = make_column_transformer(
    (OrdinalEncoder(), ['Region']),
    (StandardScaler(), ['Price']),
    (OneHotEncoder(), ['Country']))
print(ct)

ColumnTransformer(transformers=[('ordinalencoder', OrdinalEncoder(),
                                 ['Region']),
                                ('standardscaler', StandardScaler(), ['Price']),
                                ('onehotencoder', OneHotEncoder(),
                                 ['Country'])])


In [5]:
X = df_wine[['Country', 'Price', 'Region']]
y = df_wine['Rating']

In [6]:
# Создание пайплайна с фиксированным random_state
pipeline = Pipeline([
    ('ct', ct),
    ('rf', RandomForestRegressor(random_state=42))
])

pipeline.fit(X, y)

Pipeline(steps=[('ct',
                 ColumnTransformer(transformers=[('ordinalencoder',
                                                  OrdinalEncoder(),
                                                  ['Region']),
                                                 ('standardscaler',
                                                  StandardScaler(), ['Price']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['Country'])])),
                ('rf', RandomForestRegressor(random_state=42))])

In [7]:
df_wine_test = pd.read_csv('data/Red_test.csv')

X_test = df_wine_test[['Country', 'Price', 'Region']]
y_test = df_wine_test['Rating']

y_pred = pipeline.predict(X_test)
print(f'Качество по RSME: {round(np.sqrt(mean_squared_error(y_test, y_pred)), 4)}')

Качество по RSME: 0.0765


Задание 6.2

Теперь попробуем изменить параметры случайного леса в пайплайне, полученном в предыдущем задании.

Измените параметр n_estimators в случайном лесу со значения по умолчанию до 200 , используя метод set_params.

В качестве ответа на задание введите в поле ниже полученный результат по метрике RMSE, округленный до четвёртого знака после запятой.

In [8]:
pipeline.set_params(rf__n_estimators = 200)
pipeline.fit(X,y)
 
df_wine_test = pd.read_csv('data/Red_test.csv')

X_test = df_wine_test[['Country', 'Price', 'Region']]
y_test = df_wine_test['Rating']

y_pred = pipeline.predict(X_test)
print(f'Качество по RSME: {round(np.sqrt(mean_squared_error(y_test, y_pred)), 4)}')

Качество по RSME: 0.0761


Задание 6.3

Теперь попробуем добавить стекинг в качестве модели в пайплайн.

Вам следует выполнить следующее:

Собрать StackingRegressor:
В качестве базовых моделей возьмите ридж-регрессию RidgeCV() и решающее дерево.
В качестве метамодели возьмите случайный лес с настройками (количество базовых моделей 10).
Все базовые модели стекинга модели должны быть с настройками по умолчанию (кроме random_state).
Зафиксировать random_state=42 (для всех моделей).
Заменить в пайплайне задачи 6.1 случайный лес на StackingRegressor.
Обучить модель на тренировочной выборке.
В качестве ответа на задание введите в поле ниже полученный результат по метрике RMSE, округлённый до второго знака после запятой.

In [11]:
estimators = [
    ('lr', RidgeCV()),
    ('dt',  DecisionTreeRegressor(random_state=42))
]
 
reg = StackingRegressor(
    estimators=estimators,
    final_estimator=RandomForestRegressor(n_estimators=10,
                                          random_state=42)
)
 

X = df_wine[['Country', 'Price', 'Region']]
y = df_wine['Rating']
 
ct = make_column_transformer(
    (OrdinalEncoder(), ['Region']),
     (StandardScaler(), ['Price']),
    (OneHotEncoder(), ['Country']),
    )
 
pipeline = Pipeline([('ct', ct), ('stack',reg) ])
pipeline.fit(X,y)
 
 
df_wine_test = pd.read_csv('data/Red_test.csv')

X_test = df_wine_test[['Country', 'Price', 'Region']]
y_test = df_wine_test['Rating']

y_pred = pipeline.predict(X_test)
print(f'Качество по RSME: {round(np.sqrt(mean_squared_error(y_test, y_pred)), 2)}')

Качество по RSME: 0.18
